In [180]:
import requests
from bs4 import BeautifulSoup
import json
import re
from nltk import word_tokenize
import pandas as pd

### Import list of names from json file:

In [4]:
CharacterNames = []
with open('characters.json') as json_file:
    data = json.load(json_file)
    for p in data['characters']:
        CharacterNames.append('characterName')

### Get webpages

In [29]:
base_url = 'https://gameofthrones.fandom.com/wiki/Stormborn'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')
# Find the links on the page
dumps = [a['href'] for a in soup_index.find_all('a') if 
         a.has_attr('href')]

### Get documents

In [162]:
url = 'https://gameofthrones.fandom.com/wiki/Stormborn'
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

output = ''
blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    # there may be more elements you don't want, such as "style", etc.
]

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

### Accesss all captions

In [163]:
AllH = soup.find_all(['h2', 'h3'])
for header in AllH:
    if header.span is not None: 
        header.span.unwrap()

AllHeaders = [header.text for header in AllH]
Pre = AllHeaders.index('Summary')
Post = AllHeaders.index('Appearances')

HeadlineName = AllHeaders[Pre+1:Post]
HeadlineName

['On Dragonstone',
 "In King's Landing",
 'At Winterfell',
 'In the Riverlands',
 'In Oldtown',
 'On the Narrow Sea']

### Access all releveant text (including captions)

In [164]:
allText = soup.find_all(['p', 'h3'])
for section in allText: 
    if section is not None:
        if section.a is not None: 
            section.a.unwrap()
        elif section.span is not None: 
            section.span.unwrap()
            
DragonStone = [t.text for t in allText]

In [172]:
CaptionIndices = []
for caption in HeadlineName: 
    CaptionIndices.append(DragonStone.index(caption))
    
CaptionIndices.append(-1)

6

In [183]:
SectionText = []
for i in range(len(CaptionIndices)-1):
    print(i)
    SectionText.append(DragonStone[CaptionIndices[i]+1:CaptionIndices[i+1]])

0
1
2
3
4
5


In [196]:
df = pd.DataFrame(columns = ["caption", "text"])
#data = SectionText, columns = HeadlineName
df['caption'] = HeadlineName
df['text'] = SectionText
df

,caption,text
0,On Dragonstone,"[During a thunderstorm at Dragonstone, Tyrion ..."
1,In King's Landing,"[In the throne room of the Red Keep, Queen Cer..."
2,At Winterfell,"[At Winterfell, Jon Snow, Sansa Stark, and Dav..."
3,In the Riverlands,[Arya Stark encounters Hot Pie at the Inn at t...
4,In Oldtown,"[At the Citadel, Archmaester Ebrose and Samwel..."
5,On the Narrow Sea,[Yara and Ellaria's ships are attacked by Euro...
